In [ ]:
import os

import kagglehub
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.metrics import accuracy_score

from howework.task6.utils.gradient_check import modify_fit_with_gradient_check
from howework.task6.utils.hyperparameters_search import grid_search_hyperparameters, create_neural_network, \
    visualize_hyperparameter_search, visualize_best_model_training
from howework.task6.utils.layer import FullyConnected, ReLU, BatchNorm
from howework.task6.utils.nn import NeuralNetwork

In [ ]:
path = kagglehub.dataset_download("zalando-research/fashionmnist")
path_to_train_csv = os.path.join(path, "fashion-mnist_train.csv")
path_to_test_csv = os.path.join(path, "fashion-mnist_test.csv")

train_df = pd.read_csv(path_to_train_csv)
test_df = pd.read_csv(path_to_test_csv)

In [ ]:
train_df.head()

In [ ]:
classes_num = len(train_df["label"].unique())
train_labels = train_df["label"].to_numpy()
train_images_normalized = train_df.drop("label", axis=1).to_numpy() / 255.0

classes_num, train_labels.shape, train_images_normalized.shape

In [ ]:
input_num = train_images_normalized.shape[1]
hidden_num = 256

nn = NeuralNetwork()
nn.build(
    FullyConnected(input_num, hidden_num),
    ReLU(),
    BatchNorm(hidden_num),
    FullyConnected(hidden_num, classes_num),
)

In [ ]:
nn.fit(train_images_normalized, train_labels)

In [ ]:
loss_history = nn.loss_history
accuracy_history = nn.accuracy_history

In [ ]:
sns.set_theme(style="whitegrid")

# Create a figure with 1 row and 2 columns
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Plot loss
sns.lineplot(ax=axes[0], data=loss_history, color='blue')
axes[0].set_title("Loss over Time")
axes[0].set_xlabel("Epoch")
axes[0].set_ylabel("Loss")

# Plot accuracy
sns.lineplot(ax=axes[1], data=accuracy_history, color='green')
axes[1].set_title("Accuracy over Time")
axes[1].set_xlabel("Epoch")
axes[1].set_ylabel("Accuracy")

plt.tight_layout()
plt.show()

In [ ]:
test_labels = test_df["label"].to_numpy()
test_images_normalized = test_df.drop("label", axis=1).to_numpy() / 255.0

accuracy = accuracy_score(test_labels, nn.predict(test_images_normalized))
accuracy

In [ ]:
original_fit = modify_fit_with_gradient_check(nn)
nn.fit(train_images_normalized, train_labels)

In [ ]:
# Run grid search
best_params, best_model, results = grid_search_hyperparameters(
    train_images_normalized, train_labels, test_images_normalized, test_labels,
    classes_num=10,  # Number of classes in your dataset
    nn_model_factory=create_neural_network
)

# Visualize results
visualize_hyperparameter_search(results, 'learning_rate')
visualize_hyperparameter_search(results, 'hidden_neurons_num')
visualize_hyperparameter_search(results, 'reg_strength')
visualize_hyperparameter_search(results, 'batch_size')
visualize_hyperparameter_search(results, 'optimizer')

# Visualize best model training history
visualize_best_model_training(best_model)